In [1]:
base_model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" #"unsloth/Llama-3.2-3B-bnb-4bit"
target_model_name = "unsloth/Llama-3.2-3B-Instruct"
model_name = "lucaelin/llama-3.2-3b-instruct-fc"

dataset_name = "lucaelin/glaive-function-calling-v2"

max_sequence_length = 16*1024
lora_rank = 32
lora_alpha = 8
learning_rate = 1e-4
weight_decay = 0.01

chat_template = "{% set loop_messages = messages %}{% for message in loop_messages %}{% set role = message['role'] %}{% if 'tool_calls' in message %}{% set text = '<tool_call>' + message['tool_calls'][0]['function']|tojson + '</tool_call>' %}{% endif %}{% if 'content' in message %}{% set text = message['content'] %}{% endif %}{% if loop.index0 == 0 and tools is defined %}{% set text = message['content'] + '\n<tools>\n' + tools|tojson + '\n</tools>' %}{% endif %}{% set content = '<|start_header_id|>' + role + '<|end_header_id|>\n\n'+ text | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}"

In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_name, # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

tokenizer = AutoTokenizer.from_pretrained(target_model_name)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.0: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.65 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_alpha,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.0 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [5]:
from unsloth.chat_templates import get_chat_template
import json

tokenizer.chat_template = chat_template

def formatting_prompts_func(example):
    messages = json.loads(example["messages"])
    tools = json.loads(example["tools"])
    text = tokenizer.apply_chat_template(messages, tools=tools, tokenize = False, add_generation_prompt = False)
    return { "text" : text, }
pass

from datasets import load_dataset
dataset = load_dataset(dataset_name, split = "train")

from unsloth.chat_templates import standardize_sharegpt
dataset = dataset.map(formatting_prompts_func, batched = False,)

In [6]:
dataset[5]["messages"]

'[{"role": "user", "content": "Hi, I need to calculate my monthly loan payment. I have a loan of $50000 with an annual interest rate of 5% and a loan term of 10 years. Can you help me with that?"}, {"role": "assistant", "content": "Of course, I can help you with that. Let me calculate it for you."}, {"role": "assistant", "tool_calls": [{"id": "call_de613d51-06c2-4d3d-b2e0-ea0f4a3092c5", "type": "function", "function": {"name": "calculate_loan_payment", "arguments": "{\\"principal\\": 50000, \\"interest_rate\\": 5, \\"loan_term\\": 10}"}}]}, {"role": "tool", "content": "{\\"monthly_payment\\": \\"$530.33\\"}", "tool_call_id": "call_de613d51-06c2-4d3d-b2e0-ea0f4a3092c5"}, {"role": "assistant", "content": "Based on the information provided, your monthly loan payment would be $530.33."}, {"role": "user", "content": "That\'s great! Can you also help me book a flight to New York next week?"}, {"role": "assistant", "content": "I\'m sorry, but as an AI, I don\'t have the capability to book fli

In [7]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHi, I need to calculate my monthly loan payment. I have a loan of $50000 with an annual interest rate of 5% and a loan term of 10 years. Can you help me with that?\n<tools>\n[{"name": "calculate_loan_payment", "description": "Calculate the monthly payment for a loan", "parameters": {"type": "object", "properties": {"principal": {"type": "number", "description": "The principal amount of the loan"}, "interest_rate": {"type": "number", "description": "The annual interest rate for the loan"}, "loan_term": {"type": "integer", "description": "The term of the loan in years"}}, "required": ["principal", "interest_rate", "loan_term"]}}]\n</tools><|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nOf course, I can help you with that. Let me calculate it for you.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n<tool_call>{"name": "calculate_loan_payment", "arguments": "{\\"principal\\": 50000, \\"interest_rate\\": 5, \

In [8]:
# Shuffle the dataset with a fixed seed
dataset = dataset.shuffle(seed=42)

# Split the dataset into 95% training and 5% validation
split_dataset = dataset.train_test_split(test_size=0.01, seed=42)
dataset = split_dataset['train']
validation_dataset = split_dataset['test']
print(f"Training dataset size: {len(dataset)}")
print(f"Validation dataset size: {len(validation_dataset)}")

Training dataset size: 111830
Validation dataset size: 1130


<a name="Train"></a>
### Train the model

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = validation_dataset,
    dataset_text_field = "text",
    max_seq_length = max_sequence_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        per_device_eval_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 10,
        eval_steps = 50,
        eval_strategy = "steps",
        learning_rate = learning_rate,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = weight_decay,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/111830 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1130 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [10]:
from unsloth.chat_templates import train_on_responses_only

instruction_template = "<|start_header_id|>user<|end_header_id|>\n\n"
response_template = "<|start_header_id|>assistant<|end_header_id|>\n\n"
trainer = train_on_responses_only(
    trainer,
    instruction_part = instruction_template,
    response_part = response_template,
)

Map:   0%|          | 0/111830 [00:00<?, ? examples/s]

Map:   0%|          | 0/1130 [00:00<?, ? examples/s]

We verify masking is actually done:

In [11]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHow can I find the kth smallest element in an array using Scala, ensuring both time and space complexity are optimized? Can you provide a Scala implementation using O(n log n) time complexity and O(1) space complexity?\nHere\'s a sample array in JSON format for reference:\n{\n "array": [5, 3, 8, 1, 9, 2, 7],\n "k": 3\n}\nPlease provide the implementation using [JSON data] format as input and [JSON data] code as output.\n<tools>\nnull\n</tools><|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSure, here\'s a Scala implementation that finds the kth smallest element in an array using the QuickSelect algorithm, which has an average time complexity of O(n) and worst-case time complexity of O(n^2), but with a few optimizations, we can bring it down to O(n log n) time complexity and O(1) space complexity:\n```\nimport scala.util.Random\nimport play.api.libs.json._\ndef findKthSmallestElement(json: JsValue): JsValue = {\n v

In [12]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                               \n\nSure, here\'s a Scala implementation that finds the kth smallest element in an array using the QuickSelect algorithm, which has an average time complexity of O(n) and worst-case time complexity of O(n^2), but with a few optimizations, we can bring it down to O(n log n) time complexity and O(1) space complexity:\n```\nimport scala.util.Random\nimport play.api.libs.json._\ndef findKthSmallestElement(json: JsValue): JsValue = {\n val array = (json  "array").as[Seq[Int]]\n val k = (json  "k").as[Int]\n if (k < 1 || k > array.length) {\n return Json.obj("error" -> "Invalid value of k")\n }\n val result = quickSelect(array, k)\n Json.obj("result" -> result)\n}\ndef quickSelect(array: Seq[Int], k: Int): Int = {\n var left = 0\n var right = array.length - 1\n while (left <= right) {\n val pivotIndex = Random.nextInt(right - left + 1) + left\n val newPivotIndex = p

<a name="Memory Stats"></a>
### Memory Stats

In [13]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.65 GB.
2.725 GB of memory reserved.


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 111,830 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 6,989
 "-____-"     Number of trainable parameters = 48,627,712


Step,Training Loss,Validation Loss
50,0.371100,0.532614
100,0.458300,0.494569
150,0.402200,0.480687
200,0.305000,0.468009
250,0.322700,0.462084
300,0.531900,0.457364
350,0.348000,0.453676
400,0.645900,0.449947
450,0.392300,0.447208
500,0.322400,0.444116


In [15]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

26510.9374 seconds used for training.
441.85 minutes used for training.
Peak reserved memory = 15.164 GB.
Peak reserved memory for training = 12.439 GB.
Peak reserved memory % of max memory = 64.118 %.
Peak reserved memory for training % of max memory = 52.596 %.


<a name="Inference"></a>
### Inference
Use the validation dataset to check model performance

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [16]:
FastLanguageModel.for_inference(model)
# Pick an example from the validation dataset
example = validation_dataset[0]

# Print the ground truth
print("Ground Truth:")
print(example["text"])

# Extract the text until the response part
instruction_part = example["text"].split(response_template)[0]

# Prepare the input for inference
inputs = tokenizer(
    instruction_part,
    return_tensors="pt",
    add_special_tokens=False,
).to("cuda")

# Run the inference
from transformers import TextStreamer
print("\nInference Result:")
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Ground Truth:
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hi, I would like to calculate my BMI. I weigh 70 kilograms and my height is 1.75 meters.
<tools>
[{"name": "calculate_bmi", "description": "Calculate Body Mass Index (BMI)", "parameters": {"type": "object", "properties": {"weight": {"type": "number", "description": "The weight in kilograms"}, "height": {"type": "number", "description": "The height in meters"}}, "required": ["weight", "height"]}}]
</tools><|eot_id|><|start_header_id|>assistant<|end_header_id|>

<tool_call>{"name": "calculate_bmi", "arguments": "{\"weight\": 70, \"height\": 1.75}"}</tool_call><|eot_id|><|start_header_id|>tool<|end_header_id|>

{"bmi": 22.86}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Your Body Mass Index (BMI) is 22.86. This is considered a healthy weight for your height.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Inference Result:
<|start_header_id|>assistant<|end_header_id|>

Sure, I can help you with t

<a name="Save"></a>
### Saving, loading finetuned models

Save the final model as LoRA adapters locally and publish to hub.

**[NOTE]** We use a trick here and rewrite the base model name so that it points to the target model instead. This increases model performance somehow.

In [17]:
model.active_peft_config.base_model_name_or_path = target_model_name
model.config.update({"base_model_name_or_path": target_model_name})
model.save_pretrained(model_name) # Local saving
tokenizer.save_pretrained(model_name)
model.push_to_hub(model_name+'-lora') # Online saving
tokenizer.push_to_hub(model_name+'-lora') # Online saving

config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/579 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/195M [00:00<?, ?B/s]

Saved model to https://huggingface.co/lucaelin/llama-3.2-3b-instruct-fc-lora


CommitInfo(commit_url='https://huggingface.co/lucaelin/llama-3.2-3b-instruct-fc-lora/commit/9243237779e18983767a56d1bd6a8ab0cfd95296', commit_message='Upload tokenizer', commit_description='', oid='9243237779e18983767a56d1bd6a8ab0cfd95296', pr_url=None, repo_url=RepoUrl('https://huggingface.co/lucaelin/llama-3.2-3b-instruct-fc-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='lucaelin/llama-3.2-3b-instruct-fc-lora'), pr_revision=None, pr_num=None)

Next, unload the trained model to free up resource for model merge and export 

In [18]:
import gc
import torch

del model
gc.collect()
torch.cuda.empty_cache()

Now load the model config we just saved which applies our LoRA adapter the target_model instead:

In [19]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,

    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2024.10.0: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.65 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Unsloth: We successfully patched the tokenizer to add a {% if add_generation_prompt %} to the chat_template.
This is not a bug, but please notify the Unsloth maintainers - thanks!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
                (b

In [20]:
# Pick an example from the validation dataset
example = validation_dataset[1]

# Print the ground truth
print("Ground Truth:")
print(example["text"])

# Extract the text until the response part
instruction_part = response_template.join(example["text"].split(response_template)[0:1])

# Prepare the input for inference
inputs = tokenizer(
    instruction_part,
    return_tensors="pt",
    add_special_tokens=False,
).to("cuda")

# Run the inference
from transformers import TextStreamer
print("\nInference Result:")
print(instruction_part)
print('---')
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Ground Truth:
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

I want to listen to some music. Can you play something for me?
<tools>
[{"name": "play_music", "description": "Play music based on genre or artist", "parameters": {"type": "object", "properties": {"genre": {"type": "string", "description": "The genre of the music"}, "artist": {"type": "string", "description": "The name of the artist"}}}}]
</tools><|eot_id|><|start_header_id|>assistant<|end_header_id|>

Of course! Do you have a specific genre or artist in mind?<|eot_id|><|start_header_id|>user<|end_header_id|>

I like pop music. Can you play something from Taylor Swift?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<tool_call>{"name": "play_music", "arguments": "{\"genre\": \"pop\", \"artist\": \"Taylor Swift\"}"}</tool_call><|eot_id|><|start_header_id|>tool<|end_header_id|>

{"status": "success", "message": "Now playing 'Love Story' by Taylor Swift"}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [21]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion

In [22]:
# Save to multiple GGUF options
model.push_to_hub_gguf(
    model_name+'-gguf',
    tokenizer,
    quantization_method = ["q4_k_m", "q8_0"],
)

make: Entering directory '/workspace/llama.cpp'
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE 
I NVCCFLA

100%|██████████| 28/28 [00:00<00:00, 62.33it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at lucaelin/llama-3.2-3b-instruct-fc-gguf into bf16 GGUF format.
The output location will be /workspace/lucaelin/llama-3.2-3b-instruct-fc-gguf/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: llama-3.2-3b-instruct-fc-gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model pa

...

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/lucaelin/llama-3.2-3b-instruct-fc-gguf
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q8_0.gguf:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/lucaelin/llama-3.2-3b-instruct-fc-gguf
